In [1]:
import pandas as pd
import numpy as np
import random
import plotly as px
import plotly_express as go

import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored
import warnings

color_pal = sns.color_palette("husl", 9)
warnings.filterwarnings('ignore')


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [10]:
df = pd.read_csv("AI modelling - DATA.csv")

# Assign new column names since they lack column names  
df.columns = ['Datetime', 'Temperature']
df

,Datetime,Temperature
0,2024-08-30 12:50:00,22.85
1,2024-08-30 12:50:00,22.85
2,2024-08-30 12:51:00,22.81
3,2024-08-30 12:51:00,22.84
4,2024-08-30 12:52:00,22.82
...,...,...
46979,2024-10-17 12:15:00,19.57
46980,2024-10-17 12:15:00,25.00
46981,2024-10-17 12:19:00,25.00
46982,2024-10-17 12:23:00,25.00


In [5]:
df.describe()

,22.85
count,46984.000000
mean,21.529035
std,42.855390
min,3.120000
25%,18.450000
50%,20.810000
75%,22.060000
max,655.350000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46984 entries, 0 to 46983
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Datetime     46984 non-null  object 
 1   Temperature  46984 non-null  float64
dtypes: float64(1), object(1)
memory usage: 734.3+ KB


# CREATE MODEL 

In [ ]:
from sklearn.model_selection import train_test_split

# Define your feature matrix (X) and target variable (y)
X = df.drop(columns=['TEMPERATURE SHT'])
y = df['TEMPERATURE SHT']

#prepare taining set
X_train, X_test, y_train, y_test = train_test_split( X ,y , random_state=1, test_size=0.20)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# Define regression models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Support Vector Regressor': SVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Random Forest Regressor': RandomForestRegressor()
}

# Train and evaluate models
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"{name}: {score}")
    except Exception as e:
        print(f"Error with {name}: {e}")